In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
adata1=sc.read('/mnt/f/pvn/outer/new_protocal/step3_recluster/cd4_8_T_nk_cell.h5ad')

In [ ]:
adata2 = adata1[adata1.obs['celltype_level2'].isin(['CD8 T cells'])]

In [ ]:
adata=adata2.raw.to_adata()

In [ ]:
sc.pp.highly_variable_genes(
    adata, n_top_genes=4000, flavor="seurat",batch_key="sample",
)

In [ ]:
adata.raw = adata.copy()

In [ ]:
highly_variable_genes = adata.var[adata.var['highly_variable']].index
hsp_genes = [gene for gene in highly_variable_genes if gene.startswith('Hsp')]
mt_genes = [gene for gene in highly_variable_genes if gene.startswith('mt-')]
rps_genes = [gene for gene in highly_variable_genes if gene.startswith('Rps') or gene.startswith('Rpl')]
print("Highly variable Hsp genes: ", hsp_genes)
print("Highly variable mt genes: ", mt_genes)
print("Highly variable rps genes: ", rps_genes)

In [ ]:
filtered_highly_variable_genes = [gene for gene in highly_variable_genes if gene not in hsp_genes and gene not in mt_genes and gene not in rps_genes]
adata.var['highly_variable'] = adata.var_names.isin(filtered_highly_variable_genes)
highly_variable_genes = adata.var['highly_variable']
print(f"Number of highly variable genes: {highly_variable_genes.sum()}")
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
sc.pp.regress_out(adata, keys=["total_counts", "pct_counts_mt","pct_counts_hsp"])
sc.pp.scale(adata, max_value=10)
sc.pp.pca(adata, n_comps=50)
import scanpy.external as sce
sce.pp.harmony_integrate(adata, key="sample")
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=50,use_rep='X_pca_harmony')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.6,key_added='leiden_res0.6')
sc.pl.umap(adata, color=["leiden_res0.6"], legend_loc="on data")

In [ ]:
annotations = {
    '0': 'Nkg7_CD8_Tex', '1': 'Nkg7_CD8_Tex', '2': 'Tcf7_CD8_Tn', '3': 'Mki67_CD8_Tcycling', '4': 'Themis_CD8_Tem', '5': 'Prf1_CD8_Tex', 
    '6': 'Isg15_CD8_Teff', '7': 'Ccr7_CD8_Tm'
}

In [ ]:
adata.obs['celltype_level3'] = adata.obs['leiden_res0.6'].map(annotations)

In [ ]:
sc.pl.umap(adata, color=["celltype_level3"], legend_loc="on data")

In [ ]:
adata.write("/mnt/f/pvn/outer/new_protocal/step3_recluster/cd8.h5ad")

In [ ]:
check_genesT=['Tcf7','Sell','Lef1',
            'Il7r','Ccr7','Cd83',
            'Themis','Stat3','Itk',
              'Mki67','Tuba1b','Tubb5',
              'Isg15','Ifi203','Isg20',
            'Prf1','Gzmb','Gzme','Nkg7',	
            'Ctla4', 'Entpd1', 'Tox', 'Cxcl13', 'Tnfrsf9',
              
            ]
adata.layers['scaled']=sc.pp.scale(adata,copy=True).X
sc.pl.matrixplot(adata,check_genesT,'celltype_level3',colorbar_title='mean z-score',
                layer='scaled',
                vmin=-2,vmax=2,cmap='RdBu_r'#,use_raw=True
                 #,standard_scale='var'
                 ,save="cd8t_gene_expression.pdf" 
                )

In [ ]:
adr_genes=['Adra2b','Adrb3','Adra1a','Adra1b','Adrb2','Adrb1','Adra2a']

In [ ]:
sc.pl.dotplot(adata,adr_genes, groupby='celltype_level3',standard_scale='var',save='cd8t_adr_gene_expression.pdf')

In [ ]:
genelist_Cytotoxicity = ['Gzma', 'Gzmb', 'Gzmh', 'Gzmk', 'Gzmh', 'Gnly', 'Prf1', 'Ifng', 'Tnf', 'Serpinb1', 'Serpinb6', 'Serpinb9', 'Ctsa', 
                         'Ctsb', 'Ctsc', 'Ctsd', 'Ctsw', 'Cst3', 'Cst7', 'Cstb', 'Lamp1', 'Lamp3', 'Capn2']
sc.tl.score_genes(adata,genelist_Cytotoxicity, ctrl_as_ref=True, ctrl_size=50, gene_pool=None, n_bins=25, 
                  score_name='score_Cytotoxicity_celltype_level3', random_state=0, copy=False, use_raw=True)
sc.pl.violin(adata,keys=['score_Cytotoxicity_celltype_level3'],groupby='celltype_level3',
    size=0,
    palette=palette,
    jitter=False,rotation=30,save="cd8t_Cytotoxicity_score_celltype.pdf" 
            )

In [ ]:
genelist_Exhaustion = ['Pdcd1', 'Layn', 'Havcr2', 'Lag3', 'Ctla4','Tigit', 'Tox', 'Vsir', 'Btla', 'Entpd1', 'Cd160', 'Lair1']
sc.tl.score_genes(adata,genelist_Exhaustion, ctrl_as_ref=True, ctrl_size=50, gene_pool=None, n_bins=25, 
                  score_name='score_exhaustion_celltype_level3', random_state=0, copy=False, use_raw=True)
sc.pl.violin(adata,keys=['score_exhaustion_celltype_level3'],groupby='celltype_level3',
    size=0,
    palette=palette,
    jitter=False,rotation=30,save="cd8t_exhaustion_score_celltype.pdf" 
            )
sc.pl.violin(adata,keys='score_exhaustion_celltype_level3',  palette=palette,rotation=30,
    size=0,
    jitter=False,groupby='Group',save="cd8t_exhaustion_score_group.pdf" )